Before applying any learning algorithm to the data, it is necessary to apply some preprocessing tasks as shown below:

In [25]:
import pandas as pd
import nltk
import string
from nltk.stem.porter import *
import unicodedata 
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import numpy as np

In [13]:
df = pd.read_csv("train.tsv",sep='\t' , encoding='utf-8')
df_test =  pd.read_csv("test.tsv",sep='\t' , encoding='utf-8' )
df.drop(["PhraseId","SentenceId"],axis=1,inplace=True) # remove useless columns
df_test.drop(["PhraseId","SentenceId"],axis=1,inplace=True) # remove useless columns
df.head()
df.shape


(2, 2)

In [14]:
df.head()

,Phrase,Sentiment
0,A series of escapades demonstrating the adage ...,1
1,A series of escapades demonstrating the adage ...,2


 1) Remove punctuation marks and transform all characters to lowercase using nlk library
 2) Remove stop words
 3) Steaming

 Sans oublier la segmentation afin de separer chaque phrase en mot l idée c que chaque mot dans une phrase va être consideré comme étant un feature
 

In [15]:
# Transformer des caractères spéciaux en ASCII + maj to minuscules ( sorte de normalisation)
df['Phrase'] = (df['Phrase'].str.lower()
              .str.decode('utf-8')
              .map(lambda x: unicodedata.normalize('NFKD', x))
              .str.encode('ascii', 'ignore'))

In [16]:
#punctionation
def remove_punctuation(s):
    s = ''.join([i for i in s if i not in set(string.punctuation)])
    return s

df['Phrase'] = df['Phrase'].apply(remove_punctuation)

In [17]:
df['Phrase'] = df.apply(lambda row: nltk.word_tokenize(row['Phrase']), axis=1)

In [18]:
stop = stopwords.words('english')
stop.extend(['will','now','today'])
df['Phrase'] = df['Phrase'].apply(lambda x: [item for item in x if item not in stop])


In [19]:
stemmer = SnowballStemmer("english")
df['Phrase'] = df["Phrase"].apply(lambda x: [stemmer.stem(y) for y in x])

In [20]:
df['Phrase'] = df[df.Phrase.map(lambda x: len(x) if isinstance(x, list) else None) != 0]
df = df.dropna()

In [21]:
df.shape

(2, 2)

In [23]:
tf = df.Phrase.apply(pd.value_counts).fillna(0)
tf

,adag,also,amount,amus,demonstr,escapad,gander,good,goos,much,none,occasion,seri,stori
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0


In [28]:
idf = np.log((len(df) + 1 ) / (tf.gt(0).sum() + 1))
idf
tfidf = tf * idf

In [29]:
tfidf

,adag,also,amount,amus,demonstr,escapad,gander,good,goos,much,none,occasion,seri,stori
0,0.0,0.405465,0.405465,0.405465,0.0,0.0,0.405465,0.0,0.0,0.405465,0.405465,0.405465,0.0,0.405465
1,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000


In [30]:
df

,Phrase,Sentiment
0,"[seri, escapad, demonstr, adag, good, goos, al...",1
1,"[seri, escapad, demonstr, adag, good, goos]",2


In [141]:
df

,Phrase,Sentiment
0,"[seri, escapad, demonstr, adag, good, goos, al...",1
1,"[seri, escapad, demonstr, adag, good, goos]",2
2,[seri],2
3,[],2
4,[seri],2
5,"[escapad, demonstr, adag, good, goos]",2
6,[],2
7,"[escapad, demonstr, adag, good, goos]",2
8,[escapad],2
9,"[demonstr, adag, good, goos]",2
